In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from pathlib import Path
#import tensorflow as tf

# Import our input dataset
data = Path('./Data/merged_clean_3Years_df.csv')
original_df = pd.read_csv(data)
original_df

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,...,reportedzipcode,stns,obs,pred,stn_lat,stn_lon,new_date_column,geometry,rolling_2days_obs,rolling_7days_obs
0,NO CHATHAM,2/21/2015,999,AE,41.7,-70.0,3,1,1/1/1955,2618.18,...,2650,14281,0.38,0.000000,41.6875,-69.9933,2/21/2015,POINT (-69.9933 41.6875),3.800000e-01,1.680000e+00
1,NEW BRAUNFELS,10/30/2015,2,AE,29.7,-98.1,3,1,12/2/2003,3309.59,...,78130,526,10.62,9.319764,29.7221,-98.1311,10/30/2015,POINT (-98.1311 29.7221),1.062000e+01,2.208000e+01
2,NEW BRAUNFELS,10/30/2015,7,AE,29.7,-98.1,2,1,7/1/2003,0.00,...,78130,526,10.62,9.319764,29.7221,-98.1311,10/30/2015,POINT (-98.1311 29.7221),1.062000e+01,2.208000e+01
3,NEW BRAUNFELS,10/30/2015,0,C,29.7,-98.1,3,1,10/17/1998,67301.33,...,78132,526,10.62,9.319764,29.7221,-98.1311,10/30/2015,POINT (-98.1311 29.7221),1.062000e+01,2.208000e+01
4,NEW BRAUNFELS,10/30/2015,999,C,29.7,-98.1,3,3,7/1/1995,0.00,...,78130,526,10.62,9.319764,29.7221,-98.1311,10/30/2015,POINT (-98.1311 29.7221),1.062000e+01,2.208000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75656,HAMMOND,4/11/2014,999,A,44.5,-75.6,2,1,1/1/1976,0.00,...,13646,17787,0.00,0.000000,44.3768,-69.2616,4/11/2014,POINT (-69.2616 44.3768),8.220000e-13,1.120000e+00
75657,ARVERNE,1/8/2014,999,A05,40.6,-73.8,3,1,1/1/1920,0.00,...,11692,14209,0.10,0.343227,42.5420,-123.4260,1/8/2014,POINT (-123.426 42.542),4.600000e-01,4.600000e-01
75658,YORKTOWN HTS,6/8/2013,999,X,41.3,-73.8,4,1,4/1/1968,3198.97,...,10598,14262,8.64,6.556328,41.2664,-73.7975,6/8/2013,POINT (-73.7975 41.2664),1.042000e+01,1.205000e+01
75659,MC GREGOR,10/26/2013,999,X,31.5,-97.3,1,1,6/28/2000,12654.47,...,76657,1561,0.00,0.000000,30.6081,-97.0317,10/26/2013,POINT (-97.0317 30.6081),-2.410000e-14,1.110000e-13


In [2]:
#cleaning up last7_df
df = original_df.copy()

df["rolling_7days_obs"] = df["rolling_7days_obs"].round(2)


#convert date columns to datetime64[ns] 
df['dateofloss'] =  pd.to_datetime(df['dateofloss'], format='%m/%d/%Y', errors='ignore')
df["originalconstructiondate"] = df["originalconstructiondate"].apply(pd.to_datetime)
#df['originalconstructiondate'] =  pd.to_datetime(df['originalconstructiondate'], format='%m/%d/%Y', errors='ignore')

#merged_df.dtypes
#split the dates into sperate columns for dd, mm and yyyy
#split date of loss
df[['lossyear','lossmonth','lossday']] = df.dateofloss.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))

#split original construction date
df[['builtyear','builtmonth','builtday']] = df.originalconstructiondate.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))


#convert split columns to floats
df['lossyear'] = df['lossyear'].astype(float)
df['lossmonth'] = df['lossmonth'].astype(float)
df['builtyear'] = df['builtyear'].astype(float)

#age of the property at the time of loss
df['propertyage'] = df['lossyear'] - df['builtyear'] 

In [3]:
#keeping chosen features and dropping all other columns from merged data

#df = merged_df.copy()
df = df.drop("reportedzipcode", axis=1)
df = df.drop("dateofloss", axis=1)
df = df.drop("elevationdifference", axis=1)
df = df.drop("latitude", axis=1)
df = df.drop("longitude", axis=1)
#df = df.drop("numberoffloorsintheinsuredbuilding", axis=1)
df = df.drop("originalconstructiondate", axis=1)
df = df.drop("reportedcity", axis=1)
df = df.drop("lossyear", axis=1)
df = df.drop("lossday", axis=1)
df = df.drop("builtyear", axis=1)
df = df.drop("builtmonth", axis=1)
df = df.drop("builtday", axis=1)
df = df.drop("stn_lat", axis=1)
df = df.drop("stn_lon", axis=1)
df = df.drop("geometry", axis=1)
df = df.drop("new_date_column", axis=1)
df = df.drop("rolling_2days_obs", axis=1)
#df = df.drop("state", axis=1)

#dropping columns that might be providing redundant information
df = df.drop("obs", axis=1)
df = df.drop("pred", axis=1)
df = df.drop("stns", axis=1)


df.head(3)
df_encoded = pd.get_dummies(df, columns=["floodzone", "occupancytype", "lossmonth", "state"])
#df_encoded = pd.get_dummies(df, columns=["numberoffloorsintheinsuredbuilding", "occupancytype"])

In [4]:
#combining common floodzones
filter_zones_A = [col for col in df_encoded if col.startswith('floodzone_A')]
filter_zones_V = [col for col in df_encoded if col.startswith('floodzone_V')]

df_encoded["Zone A"] = df_encoded[filter_zones_A].sum(axis = 1, skipna = True)
df_encoded["Zone V"] = df_encoded[filter_zones_V].sum(axis = 1, skipna = True) 

df_encoded.rename(columns={"Zone A": "floodzone_A", "Zone V": "floodzone_V"})

,numberoffloorsintheinsuredbuilding,amountpaidonbuildingclaim,rolling_7days_obs,propertyage,floodzone_A,floodzone_A01,floodzone_A02,floodzone_A03,floodzone_A04,floodzone_A05,...,state_UT,state_VA,state_VI,state_VT,state_WA,state_WI,state_WV,state_WY,floodzone_A,floodzone_V
0,3,2618.18,1.68,60.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3,3309.59,22.08,12.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,0.00,22.08,12.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,67301.33,22.08,17.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0.00,22.08,20.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75656,2,0.00,1.12,38.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
75657,3,0.00,0.46,94.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
75658,4,3198.97,12.05,45.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75659,1,12654.47,0.00,13.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_encoded = df_encoded.drop(df_encoded[filter_zones_A], axis=1)
df_encoded = df_encoded.drop(df_encoded[filter_zones_V], axis=1)
df_encoded.columns.tolist()

['numberoffloorsintheinsuredbuilding',
 'amountpaidonbuildingclaim',
 'rolling_7days_obs',
 'propertyage',
 'floodzone_B',
 'floodzone_C',
 'floodzone_D',
 'floodzone_X',
 'occupancytype_1',
 'occupancytype_2',
 'occupancytype_3',
 'occupancytype_4',
 'occupancytype_6',
 'lossmonth_1.0',
 'lossmonth_2.0',
 'lossmonth_3.0',
 'lossmonth_4.0',
 'lossmonth_5.0',
 'lossmonth_6.0',
 'lossmonth_7.0',
 'lossmonth_8.0',
 'lossmonth_9.0',
 'lossmonth_10.0',
 'lossmonth_11.0',
 'lossmonth_12.0',
 'state_AK',
 'state_AL',
 'state_AR',
 'state_AZ',
 'state_CA',
 'state_CO',
 'state_CT',
 'state_DC',
 'state_DE',
 'state_FL',
 'state_GA',
 'state_GU',
 'state_HI',
 'state_IA',
 'state_ID',
 'state_IL',
 'state_IN',
 'state_KS',
 'state_KY',
 'state_LA',
 'state_MA',
 'state_MD',
 'state_ME',
 'state_MI',
 'state_MN',
 'state_MO',
 'state_MS',
 'state_MT',
 'state_NC',
 'state_ND',
 'state_NE',
 'state_NH',
 'state_NJ',
 'state_NM',
 'state_NV',
 'state_NY',
 'state_OH',
 'state_OK',
 'state_OR',
 's

In [6]:
#df_encoded_withrain = df_encoded.copy()

indexNames = df_encoded[df_encoded['rolling_7days_obs'] < 0.5].index
 
# Delete these row indexes from dataFrame
df_encoded.drop(indexNames , inplace=True)

df_encoded.rolling_7days_obs.min()

0.5

In [7]:
y =df_encoded.copy()
X =df_encoded.copy()
X = X.drop("amountpaidonbuildingclaim", axis=1)

# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y['amountpaidonbuildingclaim'],
                                                   random_state=1)

# Train the Classifier

from sklearn.linear_model import LinearRegression


#random forest, neural networks with linear regression
model = LinearRegression() 
model.fit(X_train, y_train)

score_preci7= model.score(X_test, y_test)
print (score_preci7)

y_pred = model.predict(X_test)

import numpy as np
df_pred = pd.DataFrame({'Actual Payoffs': y_test, 'Predicted Payoffs': y_pred})
df_pred

0.08887972302623692


,Actual Payoffs,Predicted Payoffs
42890,31809.88,28376.266966
39522,5229.58,5956.929705
6708,131.32,21372.622219
8705,13601.29,15354.688648
54020,11074.04,-5359.314480
...,...,...
69457,23426.29,14488.710605
35630,31273.48,35177.248660
36369,3260.71,16703.487416
43871,0.00,21116.992831


In [8]:
#from sklearn.linear_model import PolynomialRegression

In [9]:
#generating a correlation matrix
corrMatrix = df.corr()

import seaborn as sn
sn.heatmap(corrMatrix, annot=True)

In [15]:
df_encoded.iloc[0]

numberoffloorsintheinsuredbuilding       3.00
amountpaidonbuildingclaim             2618.18
rolling_7days_obs                        1.68
propertyage                             60.00
floodzone_B                              0.00
                                       ...   
state_WI                                 0.00
state_WV                                 0.00
state_WY                                 0.00
Zone A                                   1.00
Zone V                                   0.00
Name: 0, Length: 81, dtype: float64

In [11]:
import pickle

In [17]:
pickle.dump(model, open('model.pkl','wb'))

model = pickle.load(open('model.pkl','rb'))
print(model.predict([[1302.0,
 15.62,
 12.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0]]))

[-2982457.03305806]
